In [ ]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-220a  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from builtins import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  8 12:19 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

# configs

In [ ]:
# TODO: Put your bucket name below and make sure you can access it without an error
bucket_name = 'fadlonbucket'

client = storage.Client()
blobs = client.list_blobs(bucket_name, prefix='postings_gcp')
# # print all the blobs in the bucket
# for b in blobs:
#     print(b.name)

In [ ]:
def open_gcp(file_name):
  """
    get a read bit stream to a gcp blob in the bucket.
    start with the prefix: 'postings_gcp/'
    
    Parameters:
    -----------
    file_name: string , a path to the file.    
    
    Returns:
    -----------
    stream to read from
    """
    client = storage.Client(file_name)
    bucket = client.bucket(bucket_name)
    blob = bucket.get_blob('postings_gcp/' + file_name)
    return blob.open('rb')

def read_pickle(file_name):
  """
    read a pickle file from the bucket.
    from 'postings_gcp/'

    Parameters:
    -----------
    file_name: string , name of the file.    
    
    Returns:
    -----------
    object that readed from pickle
    """
    stream = open_gcp(file_name+".pkl")
    pick = pickle.load(stream)
    stream.close()
    return pick

# Building Doc-id to Normalize Factor

In [ ]:
#path to the full wiki data
full_path = "gs://wikidata_preprocessed/*"
#load to spark
parquetFile = spark.read.parquet(full_path)
# Count number of wiki pages
parquetFile.count()

6348910

In [ ]:
def build_doc_id_to_title_dict():
    return dict(parquetFile.select("id", "title").rdd.collect())

doc_id_to_title_dic = build_doc_id_to_title_dict()

### save to bucket

In [ ]:
import pickle

name = "doc_id_to_title_dict"
with open(f"{name}.pkl", "wb") as f:
    pickle.dump(doc_id_to_title_dic, f)

client = storage.Client()
bucket = client.bucket(bucket_name)
blob_posting_locs = bucket.blob(f"postings_gcp/{name}.pkl")
blob_posting_locs.upload_from_filename(f"{name}.pkl")